In [1]:
import requests
import csv
import datetime
import sys
import pandas as pd

/Users/eliotsouthon/Desktop/POC/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
# Remplacez 'chemin/vers/votre/projet/module' par le chemin absolu réel vers le dossier "module" via la commande pwd
module_path = '/Users/eliotsouthon/Desktop/POC/Code/Modules'  
sys.path.append(module_path)
import ma_bibliotheque 


In [18]:

def get_weather_forecast_3h(lat, lon, api_key):
    url = f"https://api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}&appid={api_key}&units=metric"
    
    # Requête vers l'API
    response = requests.get(url)
    
    # Vérification du succès de la requête
    if response.status_code == 200:
        data = response.json()
        
        # Extraire les prévisions toutes les 3 heures pour les 5 jours
        forecast_data = []
        for forecast in data['list']:
            forecast_info = {
                "date": datetime.datetime.fromtimestamp(forecast['dt']).strftime('%Y-%m-%d %H:%M:%S'),
                "location": f"{lat},{lon}",
                "temperature": forecast['main']['temp'],
                "humidity": forecast['main']['humidity'],
                "wind_speed": forecast['wind']['speed']
            }
            forecast_data.append(forecast_info)
        
        return pd.DataFrame(forecast_data)
    else:
        print("Erreur lors de la récupération des données:", response.status_code)
        return None

def get_hourly_interpolated_data(lat, lon, api_key, filename='weather_forecast.csv'):
    # Étape 1 : Récupérer les prévisions 3 heures
    df= get_weather_forecast_3h(lat, lon, api_key)
    # Assurer que la colonne "date" est de type datetime
    df['date'] = pd.to_datetime(df['date'])
            
    # Utiliser la fonction d'interpolation de votre bibliothèque
    df_interpolated = ma_bibliotheque.interpolate_to_hourly(df)
            
    return df_interpolated


In [19]:
# Exemple d'utilisation
lat = 48.8566  # Latitude de Paris
lon = 2.3522   # Longitude de Paris
api_key = "36669f134b2c1c983dbba764cc582c16"  # Remplacer par ta clé API

forecast_data = get_hourly_interpolated_data(lat, lon, api_key)

forecast_data.head()

/Users/eliotsouthon/Desktop/POC/Code/Modules/ma_bibliotheque.py:181: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_hourly = df.resample('H').interpolate(method='linear')
/Users/eliotsouthon/Desktop/POC/Code/Modules/ma_bibliotheque.py:181: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df_hourly = df.resample('H').interpolate(method='linear')


,date,location,temperature,humidity,wind_speed
0,2024-11-20 19:00:00,"48.8566,2.3522",6.330000,65.000000,3.720000
1,2024-11-20 20:00:00,NaN,6.153333,63.333333,3.493333
2,2024-11-20 21:00:00,NaN,5.976667,61.666667,3.266667
3,2024-11-20 22:00:00,"48.8566,2.3522",5.800000,60.000000,3.040000
4,2024-11-20 23:00:00,NaN,5.460000,59.000000,2.686667


In [3]:
csv_file_path='/Users/eliotsouthon/Desktop/POC/Data/Data_test_model/weather_forecast_3h.csv'

In [4]:
def process_weather_3h_from_file(file_path, pays='france',pays_abreviation='fr', scale='region'):
    # 1. Lire le dataframe depuis le fichier
    df = pd.read_csv(file_path)
    
    # 2. Suppression de la colonne 'location'
    df = df.drop(columns=['location'], errors='ignore')
    
    # 3. Renommage des colonnes
    df = df.rename(columns={
        'date': 'Date',
        'temperature': 'Temperature',
        'humidity': 'Humidity',
        'wind_speed': 'Wind Force'
    })
    
    # 4. Ajouter le pays 'france'
    df = ma_bibliotheque.add_country(df, pays)
    
    # 5. Ajouter l'échelle 'region'
    df = ma_bibliotheque.add_scale(df, scale)
    
    # 6. Convertir la colonne 'Date' au format datetime
    df['Date'] = pd.to_datetime(df['Date'])
    
    # 7. Ajouter les colonnes de jours fériés et des jours de la semaine pour la France
    df = ma_bibliotheque.add_holiday_and_day_columns(df, pays_abreviation.upper())
    
    # 8. Ajouter les colonnes des mois et des heures
    df = ma_bibliotheque.add_month_and_hours(df)
    

    # Retourner le dataframe final
    return df

In [ ]:
df_processed = process_weather_data_from_file(csv_file_path)
df_processed.head()


In [ ]:
numero_region=75
pays= ma_bibliotheque.get_country_iso_numeric("france")
# Construire le nom du fichier en utilisant la première valeur de la colonne 'Code INSEE région'
nom_fichier = f"prediction_{pays}_{numero_region}.csv"

# Définir le chemin complet en joignant le chemin et le nom du fichier
chemin_complet = f"/Users/eliotsouthon/Desktop/POC/Data/Data_test_model/{nom_fichier}"

# Sauvegarder le DataFrame en CSV
df_processed.to_csv(chemin_complet, index=False, sep=';', encoding='utf-8')